# Adapting RED for demosaicing

## overview

RED assumes a forward model given by 

$$
\DeclareMathOperator{\vec}{vec}
\DeclareMathOperator{\diag}{\mathbf{diag}}
\newcommand{\by}{\mathbf{y}}
\newcommand{\bx}{\mathbf{x}}
\newcommand{\bH}{\mathbf{H}}
\newcommand{\be}{\mathbf{e}}
\newcommand{\bi}{\mathbf{i}}
\newcommand{\bW}{\mathbf{W}}
\newcommand{\bB}{\mathbf{B}}
\newcommand{\bI}{\mathbf{I}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\by = \bH\bx + \be
$$

where $\by$ is the _degraded_ image, the result of a linear degradation operator $\bH$ applied to the original image $\bx$ under the influence of some noise $\be \sim \mathcal{N}(\mathbf{0},\sigma^2 \mathbf{I})$. 

RED tries to solve image inverse problems, i.e. acquiring $\bx$ given the forward model and $\by$, by introducing a regularization term $\rho$ that is dependent on a denoiser $\bf$ satisfying some weak constraints,

$$
\rho(\bx) = \frac{1}{2} \bx^T (\bx - f(\bx))
$$

which could be interpreted as unnormalized cross-correlation between $\bx$ and the denoising residual. $\rho(\bx)$ is small when either
- $\bx$ serves as fixed-point of the denoising engine, $\bx \approx f(\bx)$
- cross-correlation of residual to the image itself is small, i.e. orthogonal to each other. 
    - This prior is motivated from the rationale that noisy image can be thought of as orthogonal protrusions relative to a manifold of noiseless images

RED model the inverse problem as solution to the following optimization problem

$$
\hat{\bx} = \underset{\bx}{\text{argmin }} \frac{1}{2\sigma^2} \norm{\bH\bx - \by}_2^2 + \lambda\rho(\bx)
$$

RED solves the problem with 
- fixed-point method
- steepest-descent
- ADMM

Specifically, RED uses the following denoiser
- __median filter__ and 
- __Non-locally Centralized Sparse Representation (NCSR)__ 
    - trained specifically for images with images contaminated by white Gaussian noise with a fixed noise-level of $\sigma=5$
    
Note, RED's result is equiavlent to those obtained using the $P^3$ framework. However, parameter tuning in RED is a lot simpler.


## RED on C2B images

Note, the RED paper follows the procedure outlined in the NCSR paper for testing performance of their method on image restoration tasks. Specifically, there are points that need to be changed for the purpose of demosaicing C2B images
- additive gaussian noise added to the degraded image for all restoration tasks
    - set $\sigma$ to obtain a range of $SNR=mean(input_im)/\sigma$ (25~45; ~35 for c2b camera)
- same with NCSR paper, restoring RGB image done by converting to YCbCr color-space, applying reconstruction algorithm on the luminance channel only, and then converting the result back to RGB domain
    - suspect, RED just follows NCSR's procedure. Guessing it might not work well on RGB color-space, i.e. ratios R/G, B/G not maintined explicitly by the denoising prior $\rho(\bx)$. However, for structured light C2B images, there is no explicit ratio relationship between channels: each channel is equally important.
    - Anyhow, just try the method on images on RGB color-space first and go from there.
- NCSR is trained in a supervised manner, specifically on images contaminated with white Gaussian noise with a fixed noise-level
    - may not be ideal when applied directly to C2B images: might need retraining
    - be good to try median filter denoiser as well
- NCSR denoiser is applied to luminance channel only
    - just use the denoiser on each channel independently
    
    
## Represent spatial subsampling as linear operator $\bH$

Given a matrix $S$ which maps spatial location to frame number. In the case of bayer `bggr` tiling over a 2x2 image,

$$
S = 
\begin{pmatrix}
1 & 2 \\
2 & 3 \\
\end{pmatrix}
$$

The corresponding $\bH'$ operator can be constructed as follows 

$$
\bH' = 
\begin{bmatrix}
    \diag(\mathbb{1}_{S==1}) \diag(\mathbb{1}_{S==2}) \diag(\mathbb{1}_{S==3})
\end{bmatrix}
=
\begin{pmatrix}
    1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1 \\
\end{pmatrix}
$$



## Problems

- the resulting demosaiced image $x$ from RED+ADMM showed mosaiced pattern
    - observed that regularization term drop very rapidly in a few iterations
    - probably because setting $x_i=0$ for as many $i$ as possible is effective in reducing the value of the regularization term
    - the reason that other reconstruction tasks such as superresolution does not experience similar problem is, i think, due to reason that each output pixel depends on 47 different input pixels whereas in demosaicing, each output pixel depends on 1 input pixel. For demosaicing, about 2/3 of pixels can be changed without affecting the $\by$ under $\bH$. These pixels could be safely set to 0 to reduce the value of regularization term. Same could not be said with other reconstruction deblurring
    - try decreasing value of $\lambda$: does not work, psnr still drops
    - try incoporate the multiplexing matrix $\bW$ into degradation operator $\bH$, in hope that more input pixels is required to compute 1 output pixel
    - try 2 pixel neighborhood
    

## Incorporating multiplexing matrix $\bW$ in degradation operator $\bH$

Consider 

$$
\by = \bH\bx + \be
$$

We have a vectorized linear operator $\bH = \bB (\bW \otimes \bI_P)$

$$
\vec(\by) = \bB \vec(\bx \bW^T) = \bB (\bW \otimes \bI_P) \vec(\bx)
\quad\text{where}\quad
\bB = 
\begin{bmatrix}
    \bH' & \mathbf{0} \\
    \mathbf{0} & \bH' \\
\end{bmatrix}
$$

where $\bH'$ is the demosaicing linear operator used previously.

We can compare the pattern of matrix $\bH'$ (prev.) and $\bH$ for $S=3,P=4$ over 2x2 image

$$
\bH = 
\begin{pmatrix}
    1& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0 \\
    0& 1& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 1& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
    0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1 \\
    0& 0& 0& 0& 1& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0& 0 \\
    0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0 \\
    0& 0& 0& 0& 0& 0& 0& 1& 0& 0& 0& 1& 0& 0& 0& 0 \\
\end{pmatrix}
$$

In the end, we try to infer $P\times S$ matrix $\bx$ given $P\times 2$ two-bucket measurements. Both of these data are readily available




## RED Performance experimentation 

Want to clarify the following
- What is performance of RED when different masks are used, how does it compare to bayer mask ?
    - regularly tiled masks is equally performant when compared to bayer mask, in terms of 
        - convergence speed
        - after convergence, the resulting reconstructed image (PSNR compared to groundtruth)
        - convergence trajectory (not explored)
    - random masks not as performant
- How does initial guess affect the convergence rate of the algorithm ?
    - initial guess is important to the convergence speed of the algorithm
    - `maxfilter` seems to be a good choice

        
The following experiments are ran with the following configurations

- $F=3$, $S=3$, $W$ is the optimal bucket multiplexing matrix
- Only 2 scenes (`bowl` and `buddha`), which took 12 hrs to finish 2 scenes.
    - summary statistics are averaged 
- `light_mode=true`, i.e. ADMM outer iteration is 100
    - in reality, the algorithm converges in ~20 iterations
- noise added after multiplexing, before demosaicing to achieve a range of SNR $\in \{25,30,35,40\}$
    - image with noise is same for fixed SNR
- `mask_types`
    - `bayer` bggr pattern 
    - `toeplitz`, i.e. `[1 2 3; 2 1 2; 3 1 2]`
    - `horz3`, horizontal repeat of `[1 2 3]`
    - `vert3` vertical repeat of `[1 2 3]'`
    - `random` sample i.i.d. $\sim Categorical([1,2, ..., F])$
- `initial_guesses`
    - `groundtruth` ground truth $\bx$
    - `maxfilter` max filter over upsampled image with zero at unknown locations
    - `bayerdemosaic` matlab `demosaic` then demultiplexed $\bx$ (bayer mosaic)
    - `zeroatunknown` upsampled image with zero at unknown locations
    - `zero` the zero vector
    - `random` sample i.i.d. from $\sim Unif[0,255]$

#### Start ->end PSNR (w.r.t. ground truth) with varying SNR
- rows are `mask_types` and columns are `initial_guesses`
- observation
    - decreasing SNR makes reconstruction harder, i.e. resulting PSNR decreases.
    - optimal value is roughly achieved regardless of __initial guesses__ and __mask types__
        - how they affect trajectory of optimization could be explored with a PSNR vs. iteration plot
    - Using `horz3`/`toeplitz` achieved slightly better PSNR consistently, when compared to other mask types.

![](assets/red_perf_table.png)

#### Convergence plot with fixed mask_types

- first, we want to know what is the best initial guess possible, that helps with convergence speed of algorithm
    - we choose from `maxfilter`, `zeroatunknown`, `zero`, and `random`
    - from below, `maxfilter` seems to be a good initial guess for RED+ADMM method.
    

#### Cost function vs. iterations
- observations
    - objective function value level off at ~50 iterations

![](assets/costfunc_plot_buddha_35.png)
    
#### PNSR vs. iterations
- observations 
    - best initial guess is `maxfilter`, yield a very good result within 10 iterations
    - `zero`/`zeroatunknown` performed worse, requirig ~40 iterations to achieve a good PNSR
    - `random` takes about ~60 iterations to achieve good PSNR

![](assets/psnr_plot_buddha_35.png)

#### Convergence plot with fixed initial guesses

- does the choice `mask_type` affect convergence to optimal value ?
    - from below, we see the RED+ADMM with `random` mask cannot converge to values that other mask with regular tiling would 
    - so then, it is expected that mask do affect how well the method is able to do reconstruction
        - naively, say binary mask, left half all 0 right half all 1, would not expect the reconstruction to guess well the missing pixels
    
![](assets/psnr_plot_fixedmask_buddha_35.png)


#### Convergence plot with a fixed initial guess  (`maxfilter`)

- although mask type do affect convergence, regularly tiled masks (other than `random`) seem to perform (almost) equally well. 
    - implies that might be difficult to find other masks that perform better bayer mosaic for $F=3$; the gain is at best marginal
        - however, should try this in a systematic way. Some combinatorial/bilevel optimization and search over the space of masks having good properties (convergence speed, resulting PSNR achieved)
    - it gives confidence that for any $F>3$, a regularly tiled neighborhood would be performant
    
![](assets/psnr_plot_fixedmask_fixedinitialguess_buddha_35.png)
